In [565]:
import mysql.connector
import warnings
import requests
import json 
import codecs
from bs4 import BeautifulSoup
import pprint
import pandas as pd 
import time
import re
import os
from statistics import mean 

In [566]:
## part 1
## 1b
#i. ip address: varchar(15)
#ii. city names: varchar
#iii. zipcode: varchar(5)
## 1c
#create database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS msba")



In [567]:
#create table
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="msba",
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

mycursor.execute("CREATE TABLE IF NOT EXISTS ip_addresses (ip VARCHAR(15), city VARCHAR(255), zip VARCHAR(5))")

In [568]:
## part 2
##2a
#API key: 6c25f849a241607a067c55057b801e60

##2b
url1 = "http://api.ipstack.com/8.8.8.8?access_key=6c25f849a241607a067c55057b801e60&output=json&fields=main"
url2 = "http://api.ipstack.com/128.120.0.25?access_key=6c25f849a241607a067c55057b801e60&output=json&fields=main"
url3 = "http://api.ipstack.com/128.32.12.14?access_key=6c25f849a241607a067c55057b801e60&output=json&fields=main"
url4 = "http://api.ipstack.com/64.165.72.144?access_key=6c25f849a241607a067c55057b801e60&output=json&fields=main"
url5 = "http://api.ipstack.com/73.71.102.213?access_key=6c25f849a241607a067c55057b801e60&output=json&fields=main"

##2c
headers = ({'User-Agent':
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56'})
json1 = requests.get(url1, headers = headers)
pprint.pprint(json1.json())
json2 = requests.get(url2)
pprint.pprint(json2.json())
json3 = requests.get(url3)
pprint.pprint(json3.json())
json4 = requests.get(url4)
pprint.pprint(json4.json())
json5 = requests.get(url5)
pprint.pprint(json5.json())


{'city': 'Mountain View',
 'continent_code': 'NA',
 'continent_name': 'North America',
 'country_code': 'US',
 'country_name': 'United States',
 'ip': '8.8.8.8',
 'latitude': 37.38801956176758,
 'longitude': -122.07431030273438,
 'region_code': 'CA',
 'region_name': 'California',
 'type': 'ipv4',
 'zip': '94041'}
{'city': 'Davis',
 'continent_code': 'NA',
 'continent_name': 'North America',
 'country_code': 'US',
 'country_name': 'United States',
 'ip': '128.120.0.25',
 'latitude': 38.56296157836914,
 'longitude': -121.81600952148438,
 'region_code': 'CA',
 'region_name': 'California',
 'type': 'ipv4',
 'zip': '95616'}
{'city': 'Berkeley',
 'continent_code': 'NA',
 'continent_name': 'North America',
 'country_code': 'US',
 'country_name': 'United States',
 'ip': '128.32.12.14',
 'latitude': 37.87459945678711,
 'longitude': -122.25466918945312,
 'region_code': 'CA',
 'region_name': 'California',
 'type': 'ipv4',
 'zip': '94720'}
{'city': 'West Sacramento',
 'continent_code': 'NA',
 'con

In [569]:
##2d
doc1 = json.loads(json1.content)
doc2 = json.loads(json2.content)
doc3 = json.loads(json3.content)
doc4 = json.loads(json4.content)
doc5 = json.loads(json5.content)
doc = [doc1, doc2, doc3, doc4, doc5]
for i in range(0,5):
    print(doc[i]["city"])
    print(doc[i]["zip"])
    

Mountain View
94041
Davis
95616
Berkeley
94720
West Sacramento
95819
Walnut Creek
94597


In [570]:
##2e 
ips = pd.DataFrame(columns = ['ip', 'city', 'zip']) 
for i in range(0,5):
    ips.loc[i,'ip'] = doc[i]["ip"]
    ips.loc[i,'city'] = doc[i]["city"]
    ips.loc[i,'zip'] = doc[i]["zip"]

print(ip)

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="msba",
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

for i,row in ips.iterrows():
    sql = "INSERT INTO ip_addresses (ip, city, zip) VALUES (%s, %s, %s)"
    mycursor.execute(sql, tuple(row))

mydb.commit()





              ip             city    zip
0        8.8.8.8    Mountain View  94041
1   128.120.0.25            Davis  95616
2   128.32.12.14         Berkeley  94720
3  64.165.72.144  West Sacramento  95819
4  73.71.102.213     Walnut Creek  94597


In [577]:
### part 3
##3.a
##page number variable is "_pgn"

##3.b
user_agent = {'User-agent': 'Mozilla/5.0'} 
for pgn in range(1, 11):
    time.sleep(4)  # at least 10 seconds pause
    print("Loading page " + str(pgn) + "...")
    url_pgn = "https://www.ebay.com/e/daily-deals/hiw-presidents-day-deals-white-sale?_pgn=" + str(pgn)
    response = requests.get(url_pgn, headers = user_agent).text
    soup = BeautifulSoup(response, 'lxml')
    for link in soup.select('div > div.s-item__info.clearfix > a'):
        with open('deals.txt','a') as file:
            file.write(link.get('href')+"\n")

Loading page 1...
Loading page 2...
Loading page 3...
Loading page 4...
Loading page 5...
Loading page 6...
Loading page 7...
Loading page 8...
Loading page 9...
Loading page 10...


In [578]:
##3.c
#ignore warnings
warnings.filterwarnings("ignore")          
os.makedirs(os.path.dirname("./deals/"), exist_ok=True)
file1 = open('deals.txt', 'r')
Lines = file1.readlines()
for i,line in enumerate(Lines):
    id = re.search("/itm/.+?/([0-9]+)?", Lines[i]).group(1)
    filename = "./deals/" + id + ".html"
    time.sleep(4)
    response = requests.get(Lines[i], headers = user_agent)
    print(i)
    with open(filename, "w") as file: 
        file.write(response.text)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215


In [594]:
##3.d
#ignore warnings
deals = pd.DataFrame(columns = ['sellername', 'sellerscore', 'itemprice', 'listprice','itemsold', 'title', 'retrn','shipping','condition']) 
warnings.filterwarnings("ignore")
for i,line in enumerate(Lines):
    id = re.search("/itm/.+?/([0-9]+)?", Lines[i]).group(1)
    filename = "./deals/" + id + ".html"
    with open(filename, "r") as file:
        text = file.read()
        soup = BeautifulSoup(text, 'html.parser') 
        deals.loc[i,'sellername'] = soup.select("div.mbg.vi-VR-margBtm3 > a > span")[0].text
        deals.loc[i,'sellerscore'] = soup.select("div.mbg.vi-VR-margBtm3 > span:nth-child(2) > a")[0].text
        deals.loc[i,'itemprice'] = re.search("\$(.+)", soup.select("#prcIsum")[0].text).group(1) if soup.select("#prcIsum")!=[] else re.search("\$(.+)", soup.select("#mm-saleDscPrc")[0].text).group(1)
        deals.loc[i,'listprice'] = None if re.search("US \$(.+)<", str(soup.select("#orgPrc"))) is None else re.search("US \$(.+)<", str(soup.select("#orgPrc"))).group(1)
        deals.loc[i,'itemsold'] = re.search("(.+) sold", soup.select("span.soldwithfeedback > span > a")[0].text).group(1) if soup.select("span.soldwithfeedback > span > a") != [] else None
        deals.loc[i,'title'] = soup.select("#itemTitle > span")[0].nextSibling
        deals.loc[i,'retrn'] = soup.select("#vi-ret-accrd-txt")[0].text
        deals.loc[i,'shipping'] =  soup.select("#fshippingCost > span")[0].text if soup.select("#fshippingCost > span") != [] else soup.select("#shSummary > div.fnfvar0.fnfadjust > span")[0].text
        deals.loc[i,'condition'] = soup.select("#vi-itm-cond")[0].text
        print (i)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215


In [595]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(deals)

                    sellername sellerscore  itemprice listprice itemsold  \
0                    cozyarray      219902       9.99     87.99  181,451   
1                    cozyarray      219902      20.99     89.99   22,178   
2                    cozyarray      219902      13.99     89.99   21,138   
3                    cozyarray      219902       5.99     29.99   27,877   
4                  utopiadeals       90683   20.49/ea      None    5,864   
5                    cozyarray      219902       8.99     39.99    8,333   
6                  utopiadeals       90683   11.00/ea      None    3,546   
7                    cozyarray      219902      14.99     99.99    6,821   
8         homecollection_deals       11460      22.50      None    4,257   
9                    cozyarray      219902       6.99     69.99   21,679   
10                   cozyarray      219902      10.99     24.99    4,655   
11                   cozyarray      219902      13.99     69.99     None   
12          

In [596]:
## change the return column into binary 
for i in deals.index:
    if re.search("not accept", deals.loc[i,'retrn'])is None:
        deals.at[i, 'retrn'] = True
    else:
        deals.at[i, 'retrn'] = False
##change list price 
for i in deals.index:
    if deals.loc[i, "listprice"] is not None:
        if re.search("/ea", str(deals.loc[i, "listprice"])) is not None:
            deals.loc[i, "listprice"] = int(float(re.search("(.+)/ea", deals.loc[i, "listprice"]).group(1))*100)
        else:
            deals.loc[i, "listprice"] = int(float(deals.loc[i, "listprice"])*100)
##change item price 
for i in deals.index:
    if deals.loc[i, "itemprice"] is not None:
        if re.search("/ea", str(deals.loc[i, "itemprice"])) is not None:
            deals.loc[i, "itemprice"] = int(float(re.search("(.+)/ea", deals.loc[i, "itemprice"]).group(1))*100)
        else:
            deals.loc[i, "itemprice"] = int(float(deals.loc[i, "itemprice"])*100)
            
##change items sold into number
for i in deals.index:
    if deals.loc[i, "itemsold"] is not None:
        deals.loc[i,"itemsold"] = int(deals.loc[i,'itemsold'].replace(",", ""))

##change title to string
deals['title'] = deals['title'].astype(str)
    

In [597]:
##3.e
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="msba",
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

mycursor.execute("CREATE TABLE IF NOT EXISTS deals (sellername VARCHAR(255), sellerscore INT, itemprice INT, listprice INT, itemsold INT, title LONGTEXT, retrn BOOLEAN, shipping VARCHAR(255), conditions VARCHAR(255) )")

##INSERT
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="msba",
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

for i,row in deals.iterrows():
    sql = "INSERT INTO deals (sellername, sellerscore, itemprice, listprice, itemsold, title, retrn, shipping, conditions) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    mycursor.execute(sql, tuple(row))

mydb.commit()



In [598]:
print(pd.crosstab(index = deals['condition'], columns = 'count'))

col_0          count
condition           
New               43
New with tags    173


In [599]:
## 3.e

no_list = deals['listprice'].isna()
new_cond = deals['condition'] == 'New'
yes_list = deals['listprice'].notnull()
tag_cond = deals['condition'] == "New with tags"

#New and no list price
subset1 = deals.loc[no_list & new_cond, ['sellerscore', 'itemprice', 'itemsold']].astype('float')
subset1['itemprice'] = subset1['itemprice']/100
subset1['itemsold'] = subset1['itemsold'].fillna(0)
print(subset1.describe())

#New and with list price
subset2 = deals.loc[yes_list & new_cond, ['sellerscore', 'itemprice', 'listprice','itemsold']].astype('float')
subset2['itemprice'] = subset2['itemprice']/100
subset2['listprice'] = subset2['listprice']/100
subset2['itemsold'] = subset2['itemsold'].fillna(0)
print(subset2.describe())

#New-with-tag and no list price
subset3 = deals.loc[no_list & tag_cond, ['sellerscore', 'itemprice', 'itemsold']].astype('float')
subset3['itemprice'] = subset3['itemprice']/100
subset3['itemsold'] = subset3['itemsold'].fillna(0)
print(subset3.describe())

#New-witg-tag and with list price
subset4 = deals.loc[yes_list & tag_cond, ['sellerscore', 'itemprice', 'listprice', 'itemsold']].astype('float')
subset4['itemprice'] = subset4['itemprice']/100
subset4['listprice'] = subset4['listprice']/100
subset4['itemsold'] = subset4['itemsold'].fillna(0)
print(subset4.describe())

         sellerscore  itemprice     itemsold
count      28.000000  28.000000    28.000000
mean    53538.357143  16.097857  1715.607143
std     47875.189462  11.347271  2020.974927
min     20537.000000   5.500000    59.000000
25%     20537.000000   6.990000    72.000000
50%     22365.000000  12.990000  1260.000000
75%     82798.000000  19.232500  2779.000000
max    146983.000000  39.990000  6273.000000
        sellerscore  itemprice   listprice      itemsold
count     15.000000  15.000000   15.000000     15.000000
mean   74842.200000  21.285333   61.403333   2931.866667
std     4835.565401  20.799403   97.856739   7070.813763
min    65499.000000   9.990000   23.390000      0.000000
25%    77178.000000  12.990000   24.950000    100.000000
50%    77178.000000  19.480000   28.740000    406.000000
75%    77178.000000  19.980000   35.990000   2832.000000
max    77178.000000  94.990000  405.950000  28135.000000
         sellerscore   itemprice     itemsold
count     107.000000  107.000000   1